In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.metrics
import seaborn as sns
import sys
import os
sys.path.append('..')
import util.bq_util

## Load up some data

In [5]:
bq, gcs = util.bq_util.get_authed_clients()

Authenticating with BigQuery and GCS...
Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=124072386181-qqedvnl36ver0khc3pmqbh4bevlh58qd.apps.googleusercontent.com&access_type=offline

Enter verification code: 4/NwX_SdAbLpRgFUQUHaam-txLfdM0O7SQypvr3juM33M
Authentication successful.


In [8]:
#bq.tables().list(projectId="khanacademy.org:deductive-jet-827", datasetId="recommendations").execute()

In [24]:
bucket = "ka_users/amy/"
filename = "rec_training000000000000.csv"
util.bq_util.list_files_in_gcs_bucket(gcs, os.path.join(bucket, filename))

[u'amy/rec_training000000000000.csv']

In [25]:
util.bq_util.download_files_from_gcs(gcs, os.path.join(bucket, filename))
df = pd.read_csv(filename)

## Define preprocessing + modeling functionality

In [27]:
def feature_processing(userdf):
    """Preprocess dataframe for a single user"""
    DV = ["engaged"] # DV = did they engage with the content
    features = ["kind", 
                "title", 
                "domain", 
                "subject", 
                "topic", 
                "tutorial", 
                "num_learners_per_day", 
                "total_score",
                "content_id",
               ]
    data = userdf[DV + features].dropna()
    return data

def split_data(data): 
    """Split data into X, Y"""
    Y = data["engaged"]
    content_ids = data["content_id"].values
    features = data[[col for col in data.columns if col not in ("engaged", "content_id")]]
    # dummify anything still in categorical form
    X = pd.get_dummies(features)
    return X, Y, content_ids

def train_and_evaluate(X_feats, Y_feats, clf, print_it=False):
    """Run cross-valida"""
    X = X_feats.values
    Y = Y_feats.values
    kf = sklearn.cross_validation.StratifiedKFold(Y, n_folds=2, shuffle=True, random_state=100)
    for i, (train_index, test_index) in enumerate(kf):
        if print_it:
            print "WORKING ON FOLD %s:" %i
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, Y_train)
        predicted = clf.predict(X_test)
        probabilities = [p[1] for p in clf.predict_proba(X_test)]
        print_metrics(Y_test, predicted, probabilities, print_it=print_it)
    return Y_test, predicted, probabilities

## Train + Evaluate

## Save predictions back to BigQuery